In [ ]:
# 使用 dict() 函数创建字典
my_dict = dict(name="Alice", age=30, city="New York")
my_dict['age']

In [25]:
# lucas_kanade
import cv2
import numpy as np


In [26]:
def lucas_kanade_method(video_path):
    cap = cv2.VideoCapture(video_path)

    # ShiTomasi角点检测的参数
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    # lucas kanade光流算法的参数
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )
    # 创建一些随机的颜色
    color = np.random.randint(0, 255, (100, 3))
    # 取第一帧并在其中找到角点
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    # 返回检测到的角点坐标
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    # 创建用于绘图的掩模图像
    mask = np.zeros_like(old_frame)
    index = 0
    while True:
        index += 1
        ret, frame = cap.read()
        if not ret:
            break
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # 计算光流
        # calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts[, status[, err[, \\
        # winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]]) -> nextPts, status, err
        p1, st, err = cv2.calcOpticalFlowPyrLK(
            old_gray, frame_gray, p0, None, **lk_params
        )
        # 返回成功跟踪的特征点的位置
        # 哪些点成功跟踪（True）或失败（False）
        # 每个点的错误度量（通常是跟踪的质量或置信度）
        
        # 选择比较好的点
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        # 画出轨迹
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()  # 多维数组展开
            c, d = old.ravel()  # 多维数组展开
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
            frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        img = cv2.add(frame, mask)
        cv2.imshow("frame", img)
        cv2.imwrite(f"./out/duck_{str(index)}.jpg", img)
        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break
        if k == ord("c"):
            mask = np.zeros_like(old_frame)
        # 现在更新之前的帧和之前的点
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

In [ ]:
if __name__ == "__main__":
    video_path = "./video/slow_traffic_small.mp4"
    lucas_kanade_method(video_path)

# python lucas_kanade.py

: 